<a href="https://colab.research.google.com/github/audalsgh/20250725/blob/main/0725_traffic_light_in_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yt-dlp  # 유튜브 영상을 다운받기위한 yt-dlp 설치
!pip install ultralytics

from ultralytics import YOLO
import yt_dlp
import tempfile  # 유튜브 링크로 다운받고 빠른 처리를 위한 설정을 하는 용도

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 69.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 79.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.

In [2]:
# 필수 라이브러리 설치 (최신 yt-dlp)
!pip install -U yt-dlp

# ✅ 유튜브 주소 입력
YOUTUBE_URL = "https://www.youtube.com/watch?v=BhTLa3fdZhE&pp=ygUT64-E66Gc7KO87ZaJIOyYgeyDgQ%3D%3D"  # 테스트용: 다른 주소로 바꿔도 됨

# 저장 경로 설정
import os
os.makedirs("videos", exist_ok=True)
video_path = "videos/input.mp4"

# yt-dlp 다운로드 옵션
import yt_dlp

ydl_opts = {
    "outtmpl": video_path,                # 저장될 경로
    "format": "best[ext=mp4]/best",       # mp4 확장자 우선
    "noplaylist": True,
    "quiet": False,
    "restrictfilenames": True,
    "retries": 3,
}

# 다운로드 실행
try:
    print(f"⬇️ 유튜브 영상 다운로드 중...\n🎥 {YOUTUBE_URL}")
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([YOUTUBE_URL])
    print(f"✅ 다운로드 완료: {video_path}")
except Exception as e:
    raise RuntimeError(f"❌ 다운로드 실패: {e}")

⬇️ 유튜브 영상 다운로드 중...
🎥 https://www.youtube.com/watch?v=BhTLa3fdZhE&pp=ygUT64-E66Gc7KO87ZaJIOyYgeyDgQ%3D%3D
[youtube] Extracting URL: https://www.youtube.com/watch?v=BhTLa3fdZhE&pp=ygUT64-E66Gc7KO87ZaJIOyYgeyDgQ%3D%3D
[youtube] BhTLa3fdZhE: Downloading webpage
[youtube] BhTLa3fdZhE: Downloading tv client config
[youtube] BhTLa3fdZhE: Downloading player a1f2424c-main
[youtube] BhTLa3fdZhE: Downloading tv player API JSON
[youtube] BhTLa3fdZhE: Downloading ios player API JSON
[youtube] BhTLa3fdZhE: Downloading m3u8 information
[info] BhTLa3fdZhE: Downloading 1 format(s): 18
[download] Destination: videos/input.mp4
[download] 100% of   16.63MiB in 00:00:04 at 3.90MiB/s   
✅ 다운로드 완료: videos/input.mp4


추론에 쓰인 fps까지 계산하도록 추가함

In [4]:
from ultralytics import YOLO
import os, glob
import cv2
import time

# 🎯 1. 모델 로드
MODEL_WEIGHTS = "yolo11n.pt"   # 또는 "yolov8n.pt"
model = YOLO(MODEL_WEIGHTS)

# 📁 2. 비디오 경로
video_path = "videos/input.mp4"  # 다운로드 경로 그대로 사용

# 🎞 3. 원본 영상 FPS, 총 프레임, 길이 측정
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    raise RuntimeError(f"❌ 영상 열기 실패: {video_path}")

fps_original = cap.get(cv2.CAP_PROP_FPS)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
duration = frame_count / fps_original if fps_original else 0

print("📊 원본 영상 정보")
print(f" - FPS: {fps_original:.2f}")
print(f" - 총 프레임 수: {frame_count}")
print(f" - 길이: {duration:.2f}초")

cap.release()

# 🧠 4. traffic light 클래스 인덱스 추출
names = model.names
if isinstance(names, dict):
    tl_idx = [k for k, v in names.items() if v == 'traffic light'][0]
else:
    tl_idx = names.index('traffic light')

# ⚙️ 5. YOLO 추론 수행 (시간 측정 포함)
save_dir = "runs/detect/traffic_light"

start_time = time.time()
results = model.predict(
    source=video_path,
    conf=0.25,
    classes=[tl_idx],
    save=True,
    project="runs/detect",
    name="traffic_light",
    vid_stride=1  # 모든 프레임 사용
)
end_time = time.time()

# 📁 6. 결과 비디오 경로 탐색
out_videos = glob.glob(os.path.join(save_dir, "*.mp4"))
print("\n📁 결과 비디오:", out_videos)

# ⏱️ 7. YOLO 추론 성능 측정
elapsed = end_time - start_time
inferred_frames = len(results)
fps_yolo = inferred_frames / elapsed if elapsed > 0 else 0

print("\n⚙️ YOLO 추론 처리 정보")
print(f" - 전체 처리 시간: {elapsed:.2f}초")
print(f" - 처리된 프레임 수: {inferred_frames}")
print(f" - 평균 추론 FPS: {fps_yolo:.2f} frames/sec")

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
video 1/1 (frame 1101/6091) /content/videos/input.mp4: 384x640 1 traffic light, 9.0ms
video 1/1 (frame 1102/6091) /content/videos/input.mp4: 384x640 1 traffic light, 8.2ms
video 1/1 (frame 1103/6091) /content/videos/input.mp4: 384x640 1 traffic light, 10.6ms
video 1/1 (frame 1104/6091) /content/videos/input.mp4: 384x640 1 traffic light, 8.9ms
video 1/1 (frame 1105/6091) /content/videos/input.mp4: 384x640 1 traffic light, 9.8ms
video 1/1 (frame 1106/6091) /content/videos/input.mp4: 384x640 1 traffic light, 8.5ms
video 1/1 (frame 1107/6091) /content/videos/input.mp4: 384x640 1 traffic light, 9.9ms
video 1/1 (frame 1108/6091) /content/videos/input.mp4: 384x640 1 traffic light, 10.8ms
video 1/1 (frame 1109/6091) /content/videos/input.mp4: 384x640 1 traffic light, 8.5ms
video 1/1 (frame 1110/6091) /content/videos/input.mp4: 384x640 1 traffic light, 8.8ms
video 1/1 (frame 1111/6091) /content/videos/input.mp4: 384x640 1 traffic light, 8.6ms
video 1/1 (frame